In [6]:
import numpy as np

# Activation function (sigmoid scaled to [0.05, 0.95])
def scaled_sigmoid(x):
    return 0.9 / (1 + np.exp(-x)) + 0.05

# Derivative of the scaled sigmoid
def d_scaled_sigmoid(x):
    sx = scaled_sigmoid(x)
    return 0.9 * sx * (1 - sx) / 0.9

# Forward pass
def forward_pass(inputs, weights1, weights2):
    z1 = np.dot(inputs, weights1)
    a1 = scaled_sigmoid(z1)
    z2 = np.dot(a1, weights2)
    a2 = scaled_sigmoid(z2)
    return {
        'input': inputs,
        'z1': z1,
        'a1': a1,
        'z2': z2,
        'a2': a2
    }

# Backpropagation and weight update
def backward_pass(cache, desired_output, weights1, weights2, learning_rate=0.5):
    z1, a1, z2, a2, inputs = cache['z1'], cache['a1'], cache['z2'], cache['a2'], cache['input']
    
    # Output layer error
    error = a2 - desired_output
    delta2 = error * d_scaled_sigmoid(z2)
    
    # Hidden layer error
    delta1 = np.dot(delta2, weights2.T) * d_scaled_sigmoid(z1)
    
    # Weight updates
    grad_w2 = np.dot(a1.T, delta2)
    grad_w1 = np.dot(inputs.T, delta1)
    
    # Update weights
    weights2_new = weights2 - learning_rate * grad_w2
    weights1_new = weights1 - learning_rate * grad_w1
    
    # Clip weights to keep values in range
    weights1_new = np.clip(weights1_new, 0.05, 0.95)
    weights2_new = np.clip(weights2_new, 0.05, 0.95)
    
    return weights1_new, weights2_new

# Training loop with batching
def train_network(X, Y, epochs=100, batch_size=10):
    # Initialize weights randomly within [0.05, 0.95]
    weights1 = np.random.uniform(0.05, 0.95, (2, 2))
    weights2 = np.random.uniform(0.05, 0.95, (2, 2))
    
    for epoch in range(epochs):
        batch_w1 = []
        batch_w2 = []
        
        for i in range(batch_size):
            idx = np.random.randint(0, X.shape[0])
            x_sample = X[idx].reshape(1, -1)
            y_sample = Y[idx].reshape(1, -1)
            
            # Forward
            cache = forward_pass(x_sample, weights1, weights2)
            
            # Backward
            w1_new, w2_new = backward_pass(cache, y_sample, weights1, weights2)
            batch_w1.append(w1_new)
            batch_w2.append(w2_new)
        
        # Average the weights across batch
        weights1 = np.mean(batch_w1, axis=0)
        weights2 = np.mean(batch_w2, axis=0)
        
        # Output training progress
        print(f"Epoch {epoch+1}")
        print(f"Weights Layer 1:\n{weights1}")
        print(f"Weights Layer 2:\n{weights2}\n")
        
    return weights1, weights2

# Example use
if __name__ == "__main__":
    np.random.seed(42)

    # Example dataset (X: inputs, Y: target outputs), scaled between 0.05 and 0.95
    X = np.array([[0.1, 0.9], [0.8, 0.2]])
    Y = np.array([[0.95, 0.05], [0.05, 0.95]])

    final_w1, final_w2 = train_network(X, Y, epochs=100, batch_size=20)

    # Test on one sample
    sample_input = np.array([[0.1, 0.9]])
    activations = forward_pass(sample_input, final_w1, final_w2)

    print("=== Final Test ===")
    print(f"Input: {sample_input}")
    print(f"Z1 (pre-activation hidden): {activations['z1']}")
    print(f"A1 (post-activation hidden): {activations['a1']}")
    print(f"Z2 (pre-activation output): {activations['z2']}")
    print(f"A2 (final output): {activations['a2']}")


Epoch 1
Weights Layer 1:
[[0.38634416 0.90828617]
 [0.70844354 0.58685631]]
Weights Layer 2:
[[0.17116176 0.19497657]
 [0.07925506 0.83638947]]

Epoch 2
Weights Layer 1:
[[0.38591538 0.91003255]
 [0.70793498 0.58247045]]
Weights Layer 2:
[[0.1662311  0.18651457]
 [0.07786964 0.82978903]]

Epoch 3
Weights Layer 1:
[[0.38555726 0.91125248]
 [0.70739396 0.57692537]]
Weights Layer 2:
[[0.16831792 0.17169738]
 [0.08332401 0.81667725]]

Epoch 4
Weights Layer 1:
[[0.38498439 0.91379412]
 [0.70707118 0.57444332]]
Weights Layer 2:
[[0.15295808 0.17333288]
 [0.06976976 0.82055254]]

Epoch 5
Weights Layer 1:
[[0.38451942 0.91671065]
 [0.70676444 0.57252564]]
Weights Layer 2:
[[0.1344237  0.17815774]
 [0.06317202 0.82772231]]

Epoch 6
Weights Layer 1:
[[0.38430123 0.91800031]
 [0.70615499 0.56692903]]
Weights Layer 2:
[[0.13703314 0.16341554]
 [0.07551434 0.81476102]]

Epoch 7
Weights Layer 1:
[[0.38410007 0.91843664]
 [0.70555532 0.55972641]]
Weights Layer 2:
[[0.14990802 0.13911948]
 [0.09094776

In [1]:
import numpy as np

# ========================
# Activation functions
# ========================
def sigmoid(x):
    """Sigmoid activation: maps any real value into (0,1)"""
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    """Derivative of sigmoid given output of sigmoid"""
    return x * (1.0 - x)

# ========================
# Neural Network class
# ========================
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, lr=0.1,
                 W1_init=None, W2_init=None):
        """
        If W1_init and W2_init are provided, they should be numpy arrays of shape:
            W1_init: (input_size+1, hidden_size)  # includes bias row
            W2_init: (hidden_size+1, output_size) # includes bias row
        Otherwise randomly initialize weights in [0,1).
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = lr  # learning rate

        # Initialize or load weights
        if W1_init is not None and W2_init is not None:
            self.w1 = W1_init[:-1, :]  # shape (input_size, hidden_size)
            self.b1 = W1_init[-1:, :]  # shape (1, hidden_size)
            self.w2 = W2_init[:-1, :]  # shape (hidden_size, output_size)
            self.b2 = W2_init[-1:, :]  # shape (1, output_size)
        else:
            self.w1 = np.random.uniform(0.0, 1.0, (input_size, hidden_size))
            self.b1 = np.random.uniform(0.0, 1.0, (1, hidden_size))
            self.w2 = np.random.uniform(0.0, 1.0, (hidden_size, output_size))
            self.b2 = np.random.uniform(0.0, 1.0, (1, output_size))

    def forward(self, X):
        """Forward propagation"""
        self.z1 = np.dot(X, self.w1) + self.b1
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output):
        """Backpropagation and weight updates"""
        output_error = y - output
        output_delta = output_error * sigmoid_derivative(output)

        hidden_error = output_delta.dot(self.w2.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.a1)

        # Update weights and biases
        self.w2 += self.a1.T.dot(output_delta) * self.lr
        self.b2 += np.sum(output_delta, axis=0, keepdims=True) * self.lr
        self.w1 += X.T.dot(hidden_delta) * self.lr
        self.b1 += np.sum(hidden_delta, axis=0, keepdims=True) * self.lr

    def train(self, X, y, epochs=1000, verbose=True):
        """Train for a fixed number of epochs"""
        for epoch in range(epochs):
            # Print initial weights each epoch
            if verbose:
                print(f"--- Epoch {epoch+1}/{epochs} ---")
                print("Weights Layer 1 (w1):")
                print(self.w1)
                print("Bias Layer 1 (b1):")
                print(self.b1)
                print("Weights Layer 2 (w2):")
                print(self.w2)
                print("Bias Layer 2 (b2):")
                print(self.b2)

            # Forward + backward
            output = self.forward(X)

            # Print activations
            if verbose:
                print("z1 (pre-activation hidden):")
                print(self.z1)
                print("a1 (post-activation hidden):")
                print(self.a1)
                print("z2 (pre-activation output):")
                print(self.z2)
                print("a2 (post-activation output):")
                print(self.a2)

            # Backprop
            self.backward(X, y, output)

            # Print loss
            if verbose and (epoch + 1) % 100 == 0:
                loss = np.mean((y - output) ** 2)
                print(f"Loss: {loss:.6f}\n")

    def predict(self, X):
        """Return predicted class indices and probabilities"""
        probs = self.forward(X)
        return np.argmax(probs, axis=1), probs

    def export_snn_weights(self, prefix='NN_W_'):
        """
        Combine weights and biases into SNN format and save to .npy files:
          W1: shape (input_size+1, hidden_size)
          W2: shape (hidden_size+1, output_size)
        """
        W1 = np.vstack([self.w1, self.b1])
        W2 = np.vstack([self.w2, self.b2])
        np.save(f"{prefix}1.npy", W1)
        np.save(f"{prefix}2.npy", W2)
        print(f"Saved SNN-formatted weights: {prefix}1.npy, {prefix}2.npy")
        return W1, W2

# ========================
# Data preparation & example
# ========================
# if __name__ == "__main__":
    # Load Iris dataset and scale features to [0.05,0.95]
    # iris = datasets.load_iris()
    # X = iris.data
    # y = iris.target
    # y_encoded = np.zeros((y.size, y.max()+1))
    # y_encoded[np.arange(y.size), y] = 1
    
    # def scale_features(x):
    #     mn, mx = x.min(axis=0), x.max(axis=0)
    #     x_norm = (x - mn) / (mx - mn)
    #     return x_norm * 0.9 + 0.05

    # X_scaled = scale_features(X)
    # X_train, X_test, y_train, y_test = train_test_split(
    #     X_scaled, y_encoded, test_size=0.2,
    #     random_state=42, stratify=y
    # )

    # # Create, train, export
    # nn = NeuralNetwork(input_size=4, hidden_size=10, output_size=3, lr=0.1)
    # nn.train(X_train, y_train, epochs=1000)
    # acc = np.mean(nn.predict(X_test)[0] == np.argmax(y_test, axis=1))
    # print(f"Test Accuracy: {acc*100:.2f}%")

    # # Export in SNN-compatible format
    # W1_snn, W2_snn = nn.export_snn_weights(prefix='NN_W_')

# example of loading them here
#     W1_0 = np.load("W1.npy")
#       W2_0 = np.load("W2.npy")



In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# --- your synthetic inputs and targets ---
x0 = np.array([0.1, 0.9])
x1 = np.array([0.6, 0.7])
X = np.vstack([x0 if i % 2 == 0 else x1 for i in range(10)])  # shape (150,4)

y0 = np.array([0.95, 0.05])   # Class 0 target
y1 = np.array([0.0, 0.95])   # Class 2 target
Y = np.vstack([y0 if i % 2 == 0 else y1 for i in range(10)])  # shape (150,3)

# --- split into 80/20 train/test ---
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=[0 if i%2==0 else 1 for i in range(10)]
)

# --- build & train your network ---
nn = NeuralNetwork(input_size=2, hidden_size=2, output_size=2, lr=0.1)
nn.train(X_train, y_train, epochs=1000)

# --- evaluate on the held‐out 20% ---
y_pred, y_probs = nn.predict(X_test)

# if you want an “accuracy”‐style measure, you need class indices:
y_true_idx = np.argmax(y_test, axis=1)
y_pred_idx = y_pred
accuracy = np.mean(y_pred_idx == y_true_idx)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

W1_snn, W2_snn = nn.export_snn_weights(prefix='NN_W_')


--- Epoch 1/1000 ---
Weights Layer 1 (w1):
[[0.02002071 0.95858199]
 [0.10530154 0.25047551]]
Bias Layer 1 (b1):
[[0.09999882 0.55094374]]
Weights Layer 2 (w2):
[[0.98194731 0.36890322]
 [0.01231465 0.20250918]]
Bias Layer 2 (b2):
[[0.30042661 0.87092937]]
z1 (pre-activation hidden):
[[0.18572232 1.30142578]
 [0.19677228 0.87222989]
 [0.19677228 0.87222989]
 [0.19677228 0.87222989]
 [0.18572232 1.30142578]
 [0.18572232 1.30142578]
 [0.18572232 1.30142578]
 [0.19677228 0.87222989]]
a1 (post-activation hidden):
[[0.54629758 0.78607484]
 [0.54903495 0.70520948]
 [0.54903495 0.70520948]
 [0.54903495 0.70520948]
 [0.54629758 0.78607484]
 [0.54629758 0.78607484]
 [0.54629758 0.78607484]
 [0.54903495 0.70520948]]
z2 (pre-activation output):
[[0.84654228 1.23164768]
 [0.84823441 1.21628153]
 [0.84823441 1.21628153]
 [0.84823441 1.21628153]
 [0.84654228 1.23164768]
 [0.84654228 1.23164768]
 [0.84654228 1.23164768]
 [0.84823441 1.21628153]]
a2 (post-activation output):
[[0.6998413  0.77410683]
 

: 

In [65]:
np.save("w1.npy", nn.w1)
np.save("b1.npy", nn.b1)
np.save("w2.npy", nn.w2)
np.save("b2.npy", nn.b2)

print(nn.w1, nn.b1, nn.w2, nn.b2)

[[-2.40257277 -0.25394539 -0.96149219 -0.89614586  2.58302045  0.13876311
  -2.31420108  1.92268281  0.97686347 -0.27843057]
 [ 0.0325858  -0.19441178 -0.19954195 -0.23771653  0.34532378 -0.23813447
  -0.78544654  0.31412567  0.4422104   0.23649642]
 [ 2.77117847  0.15841422  0.96708964  1.3150534  -2.69370418 -0.20829802
   2.90278259 -1.37555797 -0.34827805  0.47227648]
 [ 1.9274688   0.11515945  1.01965938  0.97235911 -2.9576282  -0.31292552
   2.47642143 -1.63834017 -0.82248908 -0.49885525]] [[-0.88895809 -0.82863464 -0.396815   -0.4715086   1.21101224 -0.17061962
  -0.8219439   0.07770242  0.34755369 -0.44260157]] [[-2.80590062 -0.76734158  2.40506059]
 [-0.26756683 -0.36640872  0.07490121]
 [-1.05946893 -0.89097957  0.94773775]
 [-1.2111273  -0.73491921  1.02770323]
 [ 3.29446964 -0.57184223 -3.45912812]
 [-0.01217772 -0.71448122 -0.6308465 ]
 [-2.98062316 -0.92565131  2.88581609]
 [ 1.84750426 -0.55110052 -2.04794773]
 [ 0.68793264 -1.57224144 -1.17886332]
 [-0.09207141 -0.86432

In [16]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

# ========================
# Activation functions
# ========================
def sigmoid(x):
    """Sigmoid activation: maps any real value into (0,1)"""
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    """Derivative of sigmoid given output of sigmoid"""
    return x * (1.0 - x)

# ========================
# Neural Network class
# ========================
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, lr=0.1,
                 W1_init=None, W2_init=None):
        """
        If W1_init and W2_init are provided, they should be numpy arrays of shape:
            W1_init: (input_size+1, hidden_size)  # includes bias row
            W2_init: (hidden_size+1, output_size) # includes bias row
        Otherwise randomly initialize weights in [0,1).
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = lr  # learning rate

        # Initialize or load weights
        if W1_init is not None and W2_init is not None:
            # Split combined weights into w and b
            self.w1 = W1_init[:-1, :]  # shape (input_size, hidden_size)
            self.b1 = W1_init[-1:, :]  # shape (1, hidden_size)
            self.w2 = W2_init[:-1, :]  # shape (hidden_size, output_size)
            self.b2 = W2_init[-1:, :]  # shape (1, output_size)
        else:
            # Random init in [0,1)
            self.w1 = np.random.uniform(0.0, 1.0, (input_size, hidden_size))
            self.b1 = np.random.uniform(0.0, 1.0, (1, hidden_size))
            self.w2 = np.random.uniform(0.0, 1.0, (hidden_size, output_size))
            self.b2 = np.random.uniform(0.0, 1.0, (1, output_size))

    def forward(self, X):
        """Forward propagation"""
        self.z1 = np.dot(X, self.w1) + self.b1
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output):
        """Backpropagation and weight updates"""
        output_error = y - output
        output_delta = output_error * sigmoid_derivative(output)

        hidden_error = output_delta.dot(self.w2.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.a1)

        # Update weights and biases
        self.w2 += self.a1.T.dot(output_delta) * self.lr
        self.b2 += np.sum(output_delta, axis=0, keepdims=True) * self.lr
        self.w1 += X.T.dot(hidden_delta) * self.lr
        self.b1 += np.sum(hidden_delta, axis=0, keepdims=True) * self.lr

    def train(self, X, y, epochs=1000, verbose=True):
        """Train for a fixed number of epochs"""
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)
            if verbose and (epoch + 1) % 100 == 0:
                loss = np.mean((y - output) ** 2)
                print(f"Epoch {epoch+1}/{epochs} - Loss: {loss:.4f}")

    def predict(self, X):
        """Return predicted class indices and probabilities"""
        probs = self.forward(X)
        return np.argmax(probs, axis=1), probs

    def export_snn_weights(self, prefix='NN_W_'):
        """
        Combine weights and biases into SNN format and save to .npy files:
          W1: shape (input_size+1, hidden_size)
          W2: shape (hidden_size+1, output_size)
        Saves as '{prefix}1.npy' and '{prefix}2.npy'.
        Returns the combined arrays.
        """
        W1 = np.vstack([self.w1, self.b1])
        W2 = np.vstack([self.w2, self.b2])
        np.save(f"{prefix}1.npy", W1)
        np.save(f"{prefix}2.npy", W2)
        print(f"Saved SNN-formatted weights: {prefix}1.npy, {prefix}2.npy")
        return W1, W2

    def print_weights(self):
        """Utility: print raw w and b arrays"""
        print("=== w1 (input->hidden) ===")
        print(self.w1)
        print("=== b1 ===")
        print(self.b1)
        print("=== w2 (hidden->output) ===")
        print(self.w2)
        print("=== b2 ===")
        print(self.b2)

# ========================
# Data preparation & example
# ========================
if __name__ == "__main__":
    # Load Iris dataset and scale features to [0.05,0.95]
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    y_encoded = np.zeros((y.size, y.max()+1))
    y_encoded[np.arange(y.size), y] = 1
    
    def scale_features(x):
        mn, mx = x.min(axis=0), x.max(axis=0)
        x_norm = (x - mn) / (mx - mn)
        return x_norm * 0.9 + 0.05

    X_scaled = scale_features(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_encoded, test_size=0.2,
        random_state=42, stratify=y
    )

    # Create, train, export
    nn = NeuralNetwork(input_size=4, hidden_size=10, output_size=3, lr=0.1)
    nn.train(X_train, y_train, epochs=1000)
    acc = np.mean(nn.predict(X_test)[0] == np.argmax(y_test, axis=1))
    print(f"Test Accuracy: {acc*100:.2f}%")

    # Export in SNN-compatible format
    W1_snn, W2_snn = nn.export_snn_weights(prefix='NN_W_')


Epoch 100/1000 - Loss: 0.0876
Epoch 200/1000 - Loss: 0.0411
Epoch 300/1000 - Loss: 0.0147
Epoch 400/1000 - Loss: 0.0124
Epoch 500/1000 - Loss: 0.0113
Epoch 600/1000 - Loss: 0.0107
Epoch 700/1000 - Loss: 0.0103
Epoch 800/1000 - Loss: 0.0100
Epoch 900/1000 - Loss: 0.0098
Epoch 1000/1000 - Loss: 0.0096
Test Accuracy: 100.00%
Saved SNN-formatted weights: NN_W_1.npy, NN_W_2.npy
